# Accessing Data

The following code was used to generate the input data for training and testing on the SOTA GANs model, and EDSR and ESRGAN. More information for accessing data from the WIND Toolkit and NSRDB can be found at the following resources:
1. WIND Toolkit: https://www.nrel.gov/grid/wind-toolkit.html
2. NSRDB: https://nsrdb.nrel.gov/
3. Stand up your own HSDS server: https://github.com/HDFGroup/hsds
4. Use the HDF groups Kita Lab (a managed HSDS service on AWS, for higher rate limits on free trial basis): https://www.hdfgroup.org/solutions/hdf-kita/
5. HSDS Wind Examples: https://github.com/NREL/hsds-examples/blob/master/notebooks/01_WTK_introduction.ipynb
6. HSDS Solar Examples: https://github.com/NREL/hsds-examples/blob/master/notebooks/03_NSRDB_introduction.ipynb

## TFRecord Generation

In order to generate TFRecords for input for the SOTA GANs method, use the method 'generate_TFRecords().' once you have data of the shape (N_batch, height, width, [ua, va]) or (N_batch, height, width, [DNI, DHI]), you can use the following line of code to generate your TFRecords:
    
    generate_TFRecords("filename", data, mode='train', K=k)
    
This will output two TFRecords named "filename_LR.tfrecord" and "filename_HR.tfrecord", where the LR data represents the HR data, coarsened by a factor of k.

In [5]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

In [20]:
''' authors: Andrew Glaws, Karen Stengel, Ryan King
    modified by: Rupa Kurinchi-Vendhan
'''
tf.compat.v1.disable_eager_execution() # REMOVE THIS

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def downscale_image(x, K):
    tf.compat.v1.reset_default_graph()

    if x.ndim == 3:
        x = x.reshape((1, x.shape[0], x.shape[1], x.shape[2]))

    x_in = tf.compat.v1.placeholder(tf.float64, [None, x.shape[1], x.shape[2], x.shape[3]])

    weight = tf.constant(1.0/K**2, shape=[K, K, x.shape[3], x.shape[3]], dtype=tf.float64)
    downscaled = tf.compat.v1.nn.conv2d(x_in, filter=weight, strides=[1, K, K, 1], padding='SAME')

    with tf.compat.v1.Session() as sess:
        ds_out = sess.run(downscaled, feed_dict={x_in: x})

    return ds_out

def generate_TFRecords(filename, data, mode='test', K=None):
    '''
    Generate TFRecords files for model training or testing.

    Parameters:
        filename - filename for TFRecord (should by type *.tfrecord)
        data     - numpy array of size (N, h, w, c) containing data to be written to TFRecord
        mode    - if 'train', then data contains HR data that is coarsened k times
                   and both HR and LR data written to TFRecord
                   if 'test', then data contains LR data
        K        - downscaling factor, must be specified in training mode
    '''
    if mode == 'train':
        assert K is not None, 'In training mode, downscaling factor K must be specified'
        data_LR = downscale_image(data, K)
        data_HR = downscale_image(data, 1)

    LR_filename = filename + "_LR.tfrecord"
    HR_filename = filename + "_HR.tfrecord"

    with tf.compat.v1.python_io.TFRecordWriter(LR_filename) as writer:
        for j in range(data.shape[0]):
            h_LR, w_LR, c = data_LR[j, ...].shape
            features = tf.train.Features(feature={
                                    'index': _int64_feature(j),
                                'data_LR': _bytes_feature(data_LR[j, ...].tostring()),
                                    'h_LR': _int64_feature(h_LR),
                                    'w_LR': _int64_feature(w_LR),
                                        'c': _int64_feature(c)})
            example = tf.train.Example(features=features)
            writer.write(example.SerializeToString())

    with tf.compat.v1.python_io.TFRecordWriter(HR_filename) as writer:
        for j in range(data.shape[0]):
            h_LR, w_LR, c = data_HR[j, ...].shape
            features = tf.train.Features(feature={
                                    'index': _int64_feature(j),
                                'data_LR': _bytes_feature(data_HR[j, ...].tostring()),
                                    'h_LR': _int64_feature(h_LR),
                                    'w_LR': _int64_feature(w_LR),
                                        'c': _int64_feature(c)})
            example = tf.train.Example(features=features)
            writer.write(example.SerializeToString())
    writer.close()

## Downscaling

Input HR data had dimensions 100 $\times$ 100, which was coarsened using bicubic interpolation to have dimensions 25 $\times$ 25.

In [ ]:
import cv2
import numpy as np

In [ ]:
def image_change_scale(img, dimension, scale=100, interpolation=cv2.INTER_CUBIC):
    '''
    Resize image to a specificall scale of original image.
    
    Parameters:
        img       - original image (should be type numpy.ndarray)
        dimension - original image dimension (should be type tuple)
        scale     - integer scale factor to multiply the size of the original image

    Returns:
        resized_image - resized image (type numpy.ndarray)
    '''
    scale /= 100
    new_dimension = (int(dimension[1]*scale), int(dimension[0]*scale))
    resized_img = cv2.resize(img, new_dimension, interpolation=interpolation)

    return resized_img

## Wind Data

Wind velocity data is comprised of northerly and easterly wind components, denoted $v$ and $u$ respectively, calculated from 100-m height wind speed and direction. The WIND Toolkit has a spatial resolution of approximately 2-km $\times$ 2-km. The training data was sampled at a 4-hourly temporal resolution for the years 2007 to 2013, starting January 1, 2007 at 12 am.

In [31]:
%matplotlib inline
import h5pyd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

In [6]:
f = h5pyd.File("/nrel/wtk-us.h5", 'r', bucket="nrel-pds-hsds")

In [2]:
wind_timesteps = range(0, 61368, 4) # sample data in four hour intervals

In [6]:
dset_speed = f['windspeed_100m']
dset_dir = f['winddirection_100m']

In [23]:
for timestep in wind_timesteps:
    speed_HR = dset_speed[timestep,::,::]
    direction_HR = dset_dir[timestep,::,::]
    speed_HR = speed_HR[:1600,500:2100]
    direction_HR = direction_HR[:1600,500:2100]
    ua_HR = np.multiply(speed_HR, np.cos(np.radians(direction_HR+np.pi/2)))
    va_HR = np.multiply(speed_HR, np.sin(np.radians(direction_HR+np.pi/2)))
    
    h_HR = 100
    w_HR = 100
    h_LR = 20
    w_LR = 20
    
    ua_LR = ua_HR[::5, ::5]
    va_LR = va_HR[::5, ::5]
    
    ua_wind_data_HR = np.zeros(shape=(256, h_HR, w_HR))
    ua_wind_data_LR = np.zeros(shape=(256, h_LR, w_LR))
    va_wind_data_HR = np.zeros(shape=(256, h_HR, w_HR))
    va_wind_data_LR = np.zeros(shape=(256, h_LR, w_LR))
    wind_data = np.zeros((256, h_HR, h_HR, 2))
    
    idx = 0
    for row in range(16):
        for col in range(16):
            ua_wind_data_HR[idx] = ua_HR[(col*h_HR):(h_HR+col*h_HR), (row*w_HR):(w_HR+row*w_HR)]
            ua_wind_data_LR[idx] = ua_LR[(col*h_LR):(h_LR+col*h_LR), (row*w_LR):(w_LR+row*w_LR)]
            va_wind_data_HR[idx] = va_HR[(col*h_HR):(h_HR+col*h_HR), (row*w_HR):(w_HR+row*w_HR)]
            va_wind_data_LR[idx] = va_LR[(col*h_LR):(h_LR+col*h_LR), (row*w_LR):(w_LR+row*w_LR)]
            wind_data[idx] = np.dstack([ua_wind_data_HR[idx],va_wind_data_HR[idx]])
            
            ua_filename = "ua_{timestep}_{idx}.png".format(timestep=timestep, idx=idx)
            va_filename = "va_{timestep}_{idx}.png".format(timestep=timestep, idx=idx)
            
            plt.imsave("train/wind/LR/"+ua_filename, ua_wind_data_LR[idx], origin='lower', format="png")
            plt.imsave("train/wind/HR/"+ua_filename, ua_wind_data_HR[idx], origin='lower', format="png")
            plt.imsave("train/wind/LR/"+va_filename, va_wind_data_LR[idx], origin='lower', format="png")
            plt.imsave("train/wind/HR/"+va_filename, va_wind_data_HR[idx], origin='lower', format="png")
            idx += 1

## Solar Data

We consider solar irradiance data in terms of direct normal irradiance (DNI) and diffused horizontal irradiance (DHI) at a 4-km spatial resolution. The training data was obtained by randomly sampling data at an hourly temporal resolution from  6 am to 6 pm for the years 2007 to 2013, starting January 1, 2007 at 12 am.

In [ ]:
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

NSRDB offers data from 1998 to 2020. To keep consistent with the WIND Toolkit, we used test data from 2007-2013.

In [1]:
! hsls -H -v --bucket nrel-pds-hsds /nrel/nsrdb/v3/

nrel_admin                                          folder   2020-05-31 19:52:14 /nrel/nsrdb/v3/
nrel_admin                              9.8M        domain   2020-06-05 22:02:30 /nrel/nsrdb/v3/nsrdb_1998.h5
nrel_admin                              9.8M        domain   2020-06-05 22:03:08 /nrel/nsrdb/v3/nsrdb_1999.h5
nrel_admin                              9.8M        domain   2020-07-13 21:22:01 /nrel/nsrdb/v3/nsrdb_2000.h5
nrel_admin                              9.8M        domain   2020-06-05 22:03:00 /nrel/nsrdb/v3/nsrdb_2001.h5
nrel_admin                              9.8M        domain   2020-06-05 22:03:10 /nrel/nsrdb/v3/nsrdb_2002.h5
nrel_admin                              9.8M        domain   2020-06-05 22:02:30 /nrel/nsrdb/v3/nsrdb_2003.h5
nrel_admin                              9.8M        domain   2020-07-06 22:18:20 /nrel/nsrdb/v3/nsrdb_2004.h5
nrel_admin                              9.8M        domain   2020-06-05 22:02:48 /nrel/nsrdb/v3/nsrdb_2005.h5
nrel_admin             

In [18]:
def format_solar_data(f, timestep):
    '''
    Formats solar data to take the form (height, width, [DNI, DHI]).
    
    Parameters:
       f        - HDF5 file "nsrdb_{year}.h5" (mode r)
       timestep - NSRDB time index during the given year, ranging from 0 to 17520
    
    Returns:
        solar_2d - solar data of the shape (height, width, [DNI, DHI]) (type numpy.ndarray)
    '''
    dni = f['dni']
    dhi = f['dhi']

    coords = f['coordinates'][...]
    lons = coords[::10, 1]
    lats = coords[::10, 0]

    lats_unique = np.unique(lats)
    lons_unique = np.unique(lons)

    coords_2d = np.zeros((lats.shape[0], lons.shape[0], 2))
    
    for lat_i, lat in enumerate(lats_unique):
        for lon_i, lon in enumerate(lons_unique):
            coords_2d[lat_i, lon_i, 0] = lat
            coords_2d[lat_i, lon_i, 1] = lon

    solar_2d = np.zeros(coords_2d.shape)

    for c, coord in enumerate(coords):
        lat = coord[0]
        lon = coord[1]

        lat_index = np.where(lats_unique == lat)[0]
        lon_index = np.where(lons_unique == lon)[0]

        solar_2d[lat_index, lon_index, 0] = dni[timestep, c]
        solar_2d[lat_index, lon_index, 1] = dhi[timestep, c]
        
    return solar_2d

Repeat this for each year 2007-2013.

In [7]:
f = h5pyd.File("/nrel/nsrdb/v3/nsrdb_2007.h5", 'r', bucket="nrel-pds-hsds")

In [17]:
solar_timesteps = [6, 7, 8, 9, 10, 11, 12] # replace with the timesteps you need

In [130]:
for timestep in solar_timesteps:
    solar_data = format_solar_data(f, timestep)[:1600, :1600, ::] # change these values to focus on the continental US
    dni_HR = solar_data[::, ::, 0]
    dhi_HR = solar_data[::, ::, 1]
    
    h_HR = 100
    w_HR = 100
    h_LR = 20
    w_LR = 20
    
    dni_LR = dni_HR[::5, ::5]
    dhi_LR = dhi_HR[::5, ::5]
    
    dni_solar_data_HR = np.zeros(shape=(256, h_HR, w_HR))
    dni_solar_data_LR = np.zeros(shape=(256, h_LR, w_LR))
    dhi_solar_data_HR = np.zeros(shape=(256, h_HR, w_HR))
    dhi_solar_data_LR = np.zeros(shape=(256, h_LR, w_LR))

    idx=0
    for row in range(10):
        for col in range(10):
            dni_solar_data_HR[idx] = dni_HR[(col*h_HR):(h_HR+col*h_HR), (row*w_HR):(w_HR+row*w_HR)]
            dni_solar_data_LR[idx] = dni_LR[(col*h_LR):(h_LR+col*h_LR), (row*w_LR):(w_LR+row*w_LR)]
            dhi_solar_data_HR[idx] = dhi_HR[(col*h_HR):(h_HR+col*h_HR), (row*w_HR):(w_HR+row*w_HR)]
            dhi_solar_data_LR[idx] = dhi_LR[(col*h_LR):(h_LR+col*h_LR), (row*w_LR):(w_LR+row*w_LR)]
            
            dni_filename = "dni_{timestep}_{idx}".format(timestep=timestep, idx=idx)
            dhi_filename = "dhi_{timestep}_{idx}".format(timestep=timestep, idx=idx)
            
            plt.imsave("train/solar/LR/"+dni_filename+".png", dni_solar_data_LR[idx], cmap="inferno", format="png")
            plt.imsave("train/solar/HR/"+dni_filename+".png", dni_solar_data_HR[idx], cmap="inferno", format="png")
            plt.imsave("train/solar/LR/"+dhi_filename+".png", dhi_solar_data_LR[idx], cmap="inferno", format="png")
            plt.imsave("train/solar/HR/"+dhi_filename+".png", dhi_solar_data_HR[idx], cmap="inferno", format="png")
            idx += 1